### Data Summary
Import the libraries, load dataset, print shape of data, data description

In [1]:
import pandas as pd
import unicodedata
import nltk
import re
import spacy

from bs4 import BeautifulSoup
from nltk.tokenize.toktok import ToktokTokenizer

raw_data = pd.read_csv('Tweets.csv')

In [2]:
raw_data.shape

(14640, 15)

In [3]:
raw_data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
raw_data.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


In [5]:
print(pd.unique(raw_data['airline_sentiment']))
print('')
print(pd.unique(raw_data['negativereason']))
print('')
print(pd.unique(raw_data['airline']))

['neutral' 'positive' 'negative']

[nan 'Bad Flight' "Can't Tell" 'Late Flight' 'Customer Service Issue'
 'Flight Booking Problems' 'Lost Luggage' 'Flight Attendant Complaints'
 'Cancelled Flight' 'Damaged Luggage' 'longlines']

['Virgin America' 'United' 'Southwest' 'Delta' 'US Airways' 'American']


### Data Columns
1. Drop all columns except `text` and `airline_sentiment`
2. Print shape of data
3. Print first 5 rows of data

In [6]:
data = raw_data.drop(raw_data.columns.difference(['text','airline_sentiment']), 1)

In [7]:
data.shape

(14640, 2)

In [8]:
data.head(5)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


### Data Pre-Processing
1. HTML tag removal
2. Tokenization
3. Remove numbers
4. Remove special characters and punctuation
5. Convert to lowercase
6. Lemmatize or stem
7. Join words into text string
8. Print first 5 rows after pre-processing

In [9]:
# HTML tag removal
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

data['text'] = data['text'].apply(strip_html)

data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [10]:
# Remove numbers, special characters and punctuation
def remove_sc(text):
    pattern = r'[^A-Za-z\s]'
    return re.sub(pattern, '', text)

data['text'] = data['text'].apply(remove_sc)

data.head()

,airline_sentiment,text
0,neutral,VirginAmerica What dhepburn said
1,positive,VirginAmerica plus youve added commercials to ...
2,neutral,VirginAmerica I didnt today Must mean I need t...
3,negative,VirginAmerica its really aggressive to blast o...
4,negative,VirginAmerica and its a really big bad thing a...


In [11]:
# Convert to lowercase
def lower(text):
    return text.lower()

data['text'] = data['text'].apply(lower)

data.head()

,airline_sentiment,text
0,neutral,virginamerica what dhepburn said
1,positive,virginamerica plus youve added commercials to ...
2,neutral,virginamerica i didnt today must mean i need t...
3,negative,virginamerica its really aggressive to blast o...
4,negative,virginamerica and its a really big bad thing a...


In [16]:
# Tokenization
stopwords = nltk.corpus.stopwords.words('english')

def tokenize(text):
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens if token not in stopwords]
    return tokens

data['text'] = data['text'].apply(tokenize)

data.head()

,airline_sentiment,text
0,neutral,"[virginamerica, dhepburn, say]"
1,positive,"[virginamerica, plus, add, commercial, experie..."
2,neutral,"[virginamerica, nt, today, must, mean, need, t..."
3,negative,"[virginamerica, really, aggressive, blast, obn..."
4,negative,"[virginamerica, really, big, bad, thing]"


In [17]:
# Join words into text string
def join(tokens):
    return ' '.join(tokens)

data['text'] = data['text'].apply(join)

data.head()

,airline_sentiment,text
0,neutral,virginamerica dhepburn say
1,positive,virginamerica plus add commercial experience t...
2,neutral,virginamerica nt today must mean need take ano...
3,negative,virginamerica really aggressive blast obnoxiou...
4,negative,virginamerica really big bad thing


In [14]:
# Lemmatize or stem
nlp = spacy.load('en_core_web_sm')

def lemma(text):
    text = nlp(text)
    return ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])

data['text'] = data['text'].apply(lemma)

In [18]:
# Print first 5 rows after pre-processing
data.head()

,airline_sentiment,text
0,neutral,virginamerica dhepburn say
1,positive,virginamerica plus add commercial experience t...
2,neutral,virginamerica nt today must mean need take ano...
3,negative,virginamerica really aggressive blast obnoxiou...
4,negative,virginamerica really big bad thing


### Vectorization
1. Use `CountVectorizer`
2. Use `TfidfVectorizer`

### Modelling and Evaluation
Fit and evaluate model using both types of vectorization

### Conclusion
Summarize your understanding of the application of various pre-processing and vectorization and performance of your model on this dataset